# Read Data

In [ ]:
import cupy as cp

In [1]:
import csv
import pandas
import numpy as np
if __name__ == "__main__":
    dfApp = pandas.read_csv('data/app_dataframe_TRAIN.csv')
    dfArt = pandas.read_csv('data/article.csv')

    article = list(dfArt)
    article = article[1:]
    category = list(dfApp)
    category = category[3:]
    user = dfApp.user_code.tolist()
    uid = dfApp.user_id.tolist()    


In [2]:
#delete the column names and userids/usercodes, only keep the freq values
Uapp = dfApp.values
Uapp = np.delete(Uapp, np.s_[0:3], 1)

#delete the column names and userids/usercodes, only keep the dwelltime values
#     dfArt.dropna(axis=1, how='all')
# Uart = dfArt.values
# Uart = np.delete(Uart, 0, 1)


In [60]:
Uart = dfArt.values
Uart = np.delete(Uart, 0, 1)

In [5]:

#to speed up, only use the top-20 to predict
user = user[:20]
uid = uid[:20]
Uapp = Uapp[:20]
# Uart = Uart[:20]

# print(Uapp,Uart)

In [6]:
#delete articles with all nan values
dfArtP = dfArt.dropna(axis=1, how='all')
articleP = list(dfArtP)
Uartp = dfArtP.values
Uartp = np.delete(Uartp, 0, 1)
Uartp = Uartp[:20]


# Proposed Model: Collective Matrix Factorization

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Nov 14 16:10:11 2017
@author: Mika
"""

import numpy.linalg as LA
###############################################################################

"""
@INPUT:
    R     : a matrix to be factorized, dimension N x M
    I     : a matrix to be added,dimension N x C
    P     : an initial matrix of dimension N x K
    Q     : an initial matrix of dimension M x K
    S     : an initial matrix of dimension C x K
    K   : the number of latent features for R and I
    R=P * Q;
    I=P * S；
    steps : the maximum number of steps to perform the optimisation
    alpha : the learning rate
    beta  : the regularization parameter
@OUTPUT:
    the final matrices P and Q and S
"""    
def collective_matrix_factorization(R, I, P, Q, S, K, steps=5000, lamda=0.6, alpha=0.0002, beta=0.02):
    Q = Q.T
#     if S!=None:
    S = S.T        
    for step in range(steps):
        print(step)
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - cp.dot(P[i,:],Q[:,j])
#                     if I!=None:#have transfer learning#
                    for x in range(len(I[i])):
                        if I[i][x] > 0:
                            eix= I[i][x] - cp.dot(P[i,:],S[:,x])
                            for k in range(K):
                                P[i][k] = P[i][k] + alpha * (2 * lamda * eij * Q[k][j] + 2 * (1 - lamda) * eix * S[k][x] - beta * P[i][k])
                                Q[k][j] = Q[k][j] + alpha * (2 * lamda * eij * P[i][k] - beta * Q[k][j])
                                S[k][x] = S[k][x] + alpha * (2 * (1-lamda) * eix * P[i][k] - beta * S[k][x])          
#                     else:#no transfer learning#
#                         for k in xrange(K):
#                             P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
#                             Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        e_1=0
        e_2=0
        reg=0 
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e_1 = e_1 + pow(R[i][j] - cp.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        reg = reg + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
            
#         if I !=None:
        for x in range(len(I[i])):
            if I[i][x] > 0:
                e_2 = e_2 + pow(I[i][x] - cp.dot(P[i,:],S[:,x]), 2)
                for k in range(K):
                    reg = reg + (beta/2) * ( pow(S[k][x],2) )  

        e=lamda*e_1+(1-lamda)*e_2+reg
#         else:
#             e=e_1+reg
        if e < 0.001:
            break
    print('the loss is:')
    print(e)
    return P, Q.T

def RMSE(X, Y):
    S=pow(X-Y,2)
    S=S.mean()
    return cp.sqrt(S.mean)
###############################################################################

# if __name__ == "__main__":
    
#     N3 = len(Uartp)#the row of R#

#     M3 = len(Uartp[1])#the colume of user*article#
# #    print M
#     K = 5
#     C = len(Uapp[1])#the colume of user*apps#

#     P = numpy.random.rand(N,K)
#     Q = numpy.random.rand(M,K)
#     S = numpy.random.rand(C,K)
#     nP, nQ = collective_matrix_factorization(Uartp,Uapp, P, Q, S, K)

#     nR=numpy.dot(nP,nQ.T)
#     print nR

In [52]:
def fast_double_matrix_factorization(R, I, P, Q, S, K, steps=5000, lamda=0.7, alpha=0.0002, beta=0.02): 
    R[np.isnan(R)] = 0 # Use np.isnan for csv
    I[np.isnan(I)] = 0

    r_plus_mask = np.where(R>0)
    i_plus_mask = np.where(I>0)
        
    r_plus_bin = np.sign(R)    
    i_plus_bin = np.sign(I)
    
#     IX_mask = np.repeat(np.multiply(np.sum(r_plus_bin, axis=1, keepdims=True), \
#                                     np.sum(i_plus_bin, axis=0, keepdims=True)), K, axis=1)
    
    PK_mask = np.repeat(np.sum(r_plus_bin, axis=1, keepdims=True), K, axis=1)
    PK_mask += np.repeat(np.sum(i_plus_bin, axis=1, keepdims=True), K, axis=1)
    QK_mask = np.repeat(np.sum(r_plus_bin, axis=0, keepdims=True), K, axis=0)
    SK_mask = np.repeat(np.sum(i_plus_bin, axis=0, keepdims=True), K, axis=0)
    
    for step in range(steps):
        
        if step%50 == 0:
            print(step)
        
        eij = R - np.dot(P,Q.T)
        eix = I - np.dot(P,S.T)
                
        eij[np.where(R <= 0)] = 0
        eix[np.where(I <= 0)] = 0
        
#         print(QK_mask.shape)
        
        P += 2 * alpha * (lamda * np.dot(eij, Q) + (1 - lamda) * np.dot(eix, S)) - alpha * beta * np.multiply(P, PK_mask)
        Q += 2 * alpha * lamda * np.dot(eij.T, P) - alpha * beta * np.multiply(Q.T, QK_mask).T
        S += 2 * alpha * (1 - lamda) * np.dot(eix.T, P) - alpha * beta * np.multiply(S.T, SK_mask).T
        
        e1_loss = np.sum(np.power(R - np.dot(P, Q.T), 2)[r_plus_mask])
        e2_loss = np.sum(np.power(I - np.dot(P, S.T), 2)[i_plus_mask])
        e_reg = (beta/2) * (np.sum(np.multiply(np.power(P, 2), PK_mask)) + np.sum(np.multiply(np.power(Q.T, 2), QK_mask)) + np.sum(np.multiply(np.power(S.T, 2), SK_mask)))
#         print e_reg

        if step%50 == 0:
            print(lamda * e1_loss + (1-lamda) * e2_loss + e_reg)
        
        if lamda * e1_loss + (1-lamda) * e2_loss + e_reg < 0.0001:
            break

    print ('the loss is:')
    print (lamda * e1_loss + (1-lamda)*e2_loss + e_reg)
    print (e1_loss , e2_loss , e_reg)
    return P, Q

In [26]:
print(type(Uartp[0]))
print(type(Uapp[0]))

print(type(Uapp))
Uappf = Uapp.astype(float)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [53]:
N3 = len(Uartp)#the row of R#
M3 = len(Uartp[1])#the colume of user*article#
K = 5
C = len(Uapp[1])

P3 = np.random.rand(N3,K)
Q3 = np.random.rand(M3,K)
S = np.random.rand(C,K)

nP3, nQ3 = fast_double_matrix_factorization(Uartp, Uappf, P3, Q3, S, K)

nR3 = np.dot(nP3,nQ3.T)
print(nR3)

[[ 0.  0.  0. ...,  1.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  1.  2.  1.]
 [ 0.  0.  0. ...,  3.  1.  0.]
 [ 0.  0.  0. ...,  0.  2.  0.]]
0
577114.617221
50
46311.6033139
100
7727.53238013
150
1639.43981018
200
1026.21053681
250
945.721081194
300
921.72589494
350
907.371730431
400
895.876366745
450
885.59202208
500
875.902198069
550
866.521400219
600
857.300833748
650
848.153525122
700
839.021024285
750
829.857725055
800
820.62349527
850
811.280662543
900
801.793610695
950
792.130169526
1000
782.264331962
1050
772.179903914
1100
761.874619433
1150
751.36411475
1200
740.685016912
1250
729.896340351
1300
719.078475524
1350
708.329352642
1400
697.757865404
1450
687.475251292
1500
677.585672108
1550
668.177523871
1600
659.31689887
1650
651.044136103
1700
643.373689235
1750
636.296850947
1800
629.786404727
1850
623.802122811
1900
618.296157894
1950
613.217671484
2000
608.516374825
2050
604.14493557
2100
600.060383842
2150
596.224

# Baseline 1 - Matrix Factorization

In [ ]:
#######################################
#  Baseline 1 - Matrix Factorization  #
#######################################
#!/usr/bin/python
#
# Created by Albert Au Yeung (2010)
#
# An implementation of matrix factorization
#

###############################################################################

"""
@INPUT:
    R     : a matrix to be factorized, dimension N x M
    P     : an initial matrix of dimension N x K
    Q     : an initial matrix of dimension M x K
    K     : the number of latent features
    steps : the maximum number of steps to perform the optimisation
    alpha : the learning rate
    beta  : the regularization parameter
@OUTPUT:
    the final matrices P and Q
"""
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in xrange(steps):
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])
                    for k in xrange(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = numpy.dot(P,Q)
        e = 0
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)
                    for k in xrange(K):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        if e < 0.001:
            break
    return P, Q.T

###############################################################################

## Prediction 

In [ ]:
N1 = len(Uart)
M1 = len(Uart[0])
K1 = 10
P1 = numpy.random.rand(N1,K1)
Q1 = numpy.random.rand(M1,K1)
nP1, nQ1 = matrix_factorization(Uart, P1, Q1, K1)
nR1 = numpy.dot(nP, nQ.T)
print nR1

In [ ]:
nR1 = numpy.dot(nP1, nQ1.T)
print nR1
print len(nR1[0])

## Final Result

In [ ]:
topR = []
for i in range(nR1.shape[0]):
    lineR = []
    #return the index with ranked 1~5
    subRank = nR1[i].argsort()[-5:][::-1]
    print 'user #',user[i],uid[i]
#     print subRank
    for j in range(5):
        tmp = article[subRank[j]]
        lineR.append(tmp)
    print lineR
    
# print topR

In [62]:
nR = nR3.copy()

import math
topR1 = []
for i in range(nR.shape[0]):
    lineR1 = []
    lineR2 = []
    #return the index with ranked 1~5
    subRank = nR[i].argsort()[-5:][::-1]
    subRank2 = Uart[i].argsort()
    print ('user #',user[i],uid[i])
#     print subRank
    for j in range(5):
        tmp = articleP[subRank[j]]
#         tmp2 = article[subRank2[j]]
        lineR1.append(tmp)
    for k in range(nR.shape[0]):
        if math.isnan(Uart[i][subRank2[k]]):
            continue
        tmp2 = article[subRank2[k]]
        lineR2.append(tmp2)
        if len(lineR2)==5:
            break
    print (lineR1)
    print (lineR2)
    
# print topR

user # 0 00108FC5F1F3031ADD536548124A4912
['39169', '10523', '19069', '6570', '41177']
['22344']
user # 1 00310455B7B24D5CDEC892351B8D1D51
['39169', '10523', '19069', '6570', '37911']
['40351']
user # 2 00381B68ACD6CC83635CDDD9B40F0BAF
['10523', '39169', '19069', '6570', '37065']
['12897', '16891', '35968', '19071', '40613']
user # 3 003B2215B070CDB50C0819C72E208F0A
['39169', '2518', '19069', '10523', '6570']
['41089', '4367', '8597', '4283', '13976']
user # 4 003B8D414199B9C5EB7CF8C94BCC6787
['10523', '19069', '6570', '41177', '29901']
[]
user # 5 0043B1ADB92D7265FCF0B744A42AAF05
['39169', '10523', '19069', '6570', '37065']
['34968']
user # 6 00450B9C8D0055C699C03D6A0B1F6CE8
['2325', '10523', '13666', '37911', '39169']
[]
user # 7 0049D0D7726B7A1DEFB26EFE6B336B38
['10523', '39169', '19069', '6570', '41177']
[]
user # 8 004A20E76B5AFEBF06EC46D0FD33A404
['39169', '10523', '19069', '6570', '41177']
['46595', '22512', '5905', '34612', '37315']
user # 9 004A5982BCE2DECD0622672F6DFE488E
['1

In [ ]:
# topR1 = []
for i in range(nR.shape[0]):
    lineR2 = []
    #return the index with ranked 1~5
    subRank2 = Uart[i].argsort()
    print 'user #',user[i],uid[i]
    for j in range(nR.shape[0]):
        if math.isnan(Uart[i][subRank2[j]]):
            continue
        tmp2 = article[subRank2[j]]
        lineR2.append(tmp2)
        if len(lineR2)==5:
            break
    print lineR2
    
# print topR

In [ ]:
print dfArt.44347

# Baseline 2 - ?

In [ ]:
#8*5 empty
w, h = 8, 5;
Matrix = [[None for x in range(w)] for y in range(h)] 
print Matrix